In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "ahmad4922"
os.environ['KAGGLE_KEY'] = "b3bbc4af9fd19fef94e4e67bbac2a7bf"
!kaggle datasets download -d salader/dogs-vs-cats

 99% 1.05G/1.06G [00:05<00:00, 177MB/s]
100% 1.06G/1.06G [00:05<00:00, 223MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(190,190)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(190,190)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [ ]:
# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [ ]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(190,190,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(Dropout(0.2))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(Dropout(0.2))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(Dropout(0.2))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.summary()

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 188, 188, 32)      896       
                                                                 
 batch_normalization_4 (Batc  (None, 188, 188, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 94, 94, 32)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 94, 94, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 92, 92, 64)        18496     
                                                                 
 batch_normalization_5 (Batc  (None, 92, 92, 64)      

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1.0/255.0,
width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import SGD

test_it = test_datagen.flow_from_directory('/content/train',
class_mode='binary', batch_size=32, target_size=(190, 190))

test_it = test_datagen.flow_from_directory('/content/test',
class_mode='binary', batch_size=32, target_size=(190, 190))

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [16]:
history = model.fit(train_ds,epochs=10,validation_data=validation_ds)

model.save("trained_model.h5")

_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
print('> %.3f' % (acc * 100.0))

Epoch 1/10
625/625 [==============================] - 2372s 4s/step - loss: 0.7969 - accuracy: 0.5332 - val_loss: 0.6567 - val_accuracy: 0.6042
Epoch 2/10
625/625 [==============================] - 2301s 4s/step - loss: 0.6838 - accuracy: 0.5477 - val_loss: 0.6457 - val_accuracy: 0.6382
Epoch 3/10
625/625 [==============================] - 2278s 4s/step - loss: 0.6552 - accuracy: 0.5897 - val_loss: 0.7478 - val_accuracy: 0.5688
Epoch 4/10
625/625 [==============================] - 2272s 4s/step - loss: 0.6076 - accuracy: 0.6355 - val_loss: 0.5736 - val_accuracy: 0.7162
Epoch 5/10
625/625 [==============================] - 2266s 4s/step - loss: 0.5533 - accuracy: 0.7000 - val_loss: 0.7042 - val_accuracy: 0.6840
Epoch 6/10
625/625 [==============================] - 2254s 4s/step - loss: 0.5105 - accuracy: 0.7332 - val_loss: 0.5258 - val_accuracy: 0.7606
Epoch 7/10
625/625 [==============================] - 2254s 4s/step - loss: 0.4829 - accuracy: 0.7485 - val_loss: 0.4466 - val_accuracy:

<ipython-input-16-dc6418d90f2d>:5: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)


> 75.960
